# Extractor tutorial for Pic du Midi Data

The goal of this notebook is to demonstrate the capabilities of the Spectractor pipeline (still under construction). This pipeline is designed to extract a spectrum from an image given the coordinate (in pixels) of the object.

## Import the pipeline

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import os
import pandas as pd

In [26]:
if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
print('workbookDir: ' + workbookDir)
#os.chdir(workbookDir)  # If you changed the current working dir, this will take you back to the workbook dir.

workbookDir: /Users/dagoret/MacOSX/GitHub/LSST/Spectractor/notebooks


In [27]:
import sys
sys.path.append(workbookDir)
sys.path.append(os.path.dirname(workbookDir))

In [28]:
from spectractor import parameters
from spectractor.extractor.extractor import Spectractor
from spectractor.logbook import LogBook

if __name__ == "__main__":
    from argparse import ArgumentParser

    parser = ArgumentParser()
    parser.add_argument(dest="input", metavar='path', default=["tests/data/reduc_20170605_028.fits"],
                        help="Input fits file name. It can be a list separated by spaces, or it can use * as wildcard.",
                        nargs='*')
    parser.add_argument("-d", "--debug", dest="debug", action="store_true",
                        help="Enter debug mode (more verbose and plots).", default=False)
    parser.add_argument("-v", "--verbose", dest="verbose", action="store_true",
                        help="Enter verbose (print more stuff).", default=False)
    parser.add_argument("-o", "--output_directory", dest="output_directory", default="outputs/",
                        help="Write results in given output directory (default: ./outputs/).")
    parser.add_argument("-l", "--logbook", dest="logbook", default="ctiofulllogbook_jun2017_v5.csv",
                        help="CSV logbook file. (default: ctiofulllogbook_jun2017_v5.csv).")
    parser.add_argument("-c", "--config", dest="config", default="config/ctio.ini",
                        help="INI config file. (default: config.ctio.ini).")
    args = parser.parse_args()


# Configuration

## Flags

In [29]:
parameters.VERBOSE = True
parameters.DEBUG = True

## Date

In [30]:
thedate="20190214"

## Logbook

In [31]:
logbookfilename="../simple_logbook_PicDuMidi_"+thedate+".csv"
#logbookfilename="../ctiofulllogbook_jun2017_v5.csv" 

In [32]:
# good for my logbook
df=pd.read_csv(logbookfilename,sep=",", decimal=".", encoding='latin-1', header='infer')

In [33]:
df.file[0]

'T1M_20190214_234122_495_HD116405-Ronchi_Filtre_None_bin1x1.1_red.fit'

## Input filename

In [34]:
if thedate=="20190215":
    INPUTDIR="/Users/dagoret/DATA/PicDuMidiFev2019/prod_20190215"
else:
    INPUTDIR="/Users/dagoret/DATA/PicDuMidiFev2019/prod_20190214"

In [35]:
if thedate=="20190215":
    file_name="T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1_red.fit"
else:
    file_name="T1M_20190214_234122_495_HD116405-Ronchi_Filtre_None_bin1x1.1_red.fit"    

## Output directory

In [36]:
output_directory="output"

## Config

In [37]:
config="../config/picdumidi.ini"

# Run Logbook

In [38]:
logbook = LogBook(logbookfilename)

In [39]:
tag_file=file_name

In [40]:
disperser_label, target, xpos, ypos = logbook.search_for_image(tag_file)

In [41]:
disperser_label

'Ron145PDM19'

In [42]:
target

'HD116405'

In [43]:
xpos,ypos

(205, 183)

In [44]:
fullfilename=os.path.join(INPUTDIR,file_name)

In [45]:
#assert False,"STOP" 

# Run Spectractor

In [46]:
Spectractor(fullfilename,output_directory, [xpos, ypos], target, disperser_label, config,logbook=logbookfilename)

2019-03-22 18:42:46  spectractor.extractor.extractor Spectractor          INFO   
	Start SPECTRACTOR
2019-03-22 18:42:46  Image      load_LogBook         INFO   
	Load Logbook  ../simple_logbook_PicDuMidi_20190214.csv for /Users/dagoret/DATA/PicDuMidiFev2019/prod_20190214/T1M_20190214_234122_495_HD116405-Ronchi_Filtre_None_bin1x1.1_red.fit...
2019-03-22 18:42:46  Image      load_LogBook         INFO   
	Load Logbook  : date identified  20190214 for /Users/dagoret/DATA/PicDuMidiFev2019/prod_20190214/T1M_20190214_234122_495_HD116405-Ronchi_Filtre_None_bin1x1.1_red.fit...
2019-03-22 18:42:46  Image      load_LogBook         WARNING 
	Load Logbook  :  tag_date = 20190214...
2019-03-22 18:42:46  Image      load_LogBook         WARNING 
	Load Logbook  :  tag_time = 234122...
2019-03-22 18:42:46  Image      load_LogBook         WARNING 
	Load Logbook  :  tag_num = 495...
2019-03-22 18:42:46  Image      load_LogBook         WARNING 
	Load Logbook  :  tag_obj = HD116405...
2019-03-22 18:42:46  

Section: paths
x holo_dir: extractor/dispersers/	 => parameters.HOLO_DIR: /Users/dagoret/MacOSX/GitHub/LSST/Spectractor/spectractor/extractor/dispersers/	 <class 'str'>
x throughput_dir: simulation/CTIOThroughput/	 => parameters.THROUGHPUT_DIR: /Users/dagoret/MacOSX/GitHub/LSST/Spectractor/spectractor/simulation/CTIOThroughput/	 <class 'str'>
Section: instrument
x obs_name: PICDUMIDI	 => parameters.OBS_NAME: PICDUMIDI	 <class 'str'>
x obs_altitude: 2.8905	 => parameters.OBS_ALTITUDE: 2.8905	 <class 'float'>
x obs_longitude: 0 08 34.00	 => parameters.OBS_LONGITUDE: 0 08 34.00	 <class 'str'>
x obs_latitude: 42 56 11.00	 => parameters.OBS_LATITUDE: 42 56 11.00	 <class 'str'>
x obs_diameter: 1.0	 => parameters.OBS_DIAMETER: 1.0 m	 <class 'astropy.units.quantity.Quantity'>
x obs_epoch: J2000.0	 => parameters.OBS_EPOCH: J2000.0	 <class 'str'>
x obs_transmission_systematics: 0.005	 => parameters.OBS_TRANSMISSION_SYSTEMATICS: 0.005	 <class 'float'>
x obs_object_type: STAR	 => parameters.OBS_OB

TypeError: 'NoneType' object does not support item assignment